In [180]:
import numpy as np
import time
import shapely.geometry as geom
import matplotlib.pyplot as plt
import pygame

## 1.Define useful functions

In [181]:
def vertices_from_state(state, collision_rectangle):
    center = np.array(state[:2])
    angle = state[2]
    size = np.array(collision_rectangle)
    
    vertices = np.array([[1.,1.], [1.,-1.],[-1.,-1.], [-1.,1.]])
    vertices = vertices*size/2

    R = np.array(([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]]))
    vertices = np.dot(R, vertices.T).T
    vertices = vertices + center

    return vertices

def do_A_and_B_overlap(angleA, angleB, verticesA, verticesB):

    axis = []
    axis.append(np.array([np.cos(angleA), np.sin(angleA)]))
    axis.append(np.array([-np.sin(angleA), np.cos(angleA)]))
    axis.append(np.array([np.cos(angleB), np.sin(angleB)]))
    axis.append(np.array([-np.sin(angleB), np.cos(angleB)]))

    dots = []
    overlap_possible = True
    for ax in axis:
        dotA = np.dot(verticesA, ax)
        dotB = np.dot(verticesB, ax)
        if max(dotA) >= min(dotB) and max(dotB) >= min(dotA):
            overlap_possible = True
        else:
            overlap_possible = False
            break

    return overlap_possible

def create_road(total_turn_angle ,turn_point, initial_point, road_length = 100 ,resolution = 2.):

    total_turn_angle = -total_turn_angle * 3.14159 / 180.
    road_curvatures = np.concatenate([np.zeros(int(road_length*resolution*turn_point)),np.ones(10)*total_turn_angle/10.,np.zeros(int(road_length*resolution*(1-turn_point)))])

    vec = np.array([1,0])/resolution
    road_points = [initial_point]
    for c in road_curvatures:
        vec = np.dot(rotation_matrix(c), vec)
        #print(vec)
        road_points.append(road_points[-1]+ vec)
    road_points = np.array(road_points)
    return road_points
def find_index(min_ind, max_ind ,point, l):
    point = np.array(point)
    if max_ind - min_ind < 15:
        min_e = -1
        min_d = 100000
        for e in range(min_ind, max_ind):
            d = np.linalg.norm(l[e]-point)
            if d < min_d:
                min_d = d
                min_e = e
        return min_e
    split = 5
    min_e = -1
    min_d = 100000
    step = (max_ind - min_ind)//split
    for e in range(min_ind, max_ind, step):
        d = np.linalg.norm(l[e]-point)
        if d < min_d:
            min_d = d
            min_e = e

    return find_index(max(min_e - step, 0), min(min_e + step, len(l)), point, l)

def interpolate_points(points):
    max_distance = 1.
    for e in range(len(points)-1):
        p1 = points[e]
        p2 = points[e+1]
        d = np.linalg.norm(p2-p1)
        steps = int(d // max_distance) + 1
        alpha = np.linspace(0,1, steps)
        alpha = np.array([alpha, alpha]).T
        res = p1 * (1 -alpha) + p2 * alpha
        if e == 0:
            interpolated_points = res
        else:
            interpolated_points = np.concatenate([interpolated_points,res])
    return interpolated_points

def initialize_world():
    initial_point = np.array([-50,0.])
    maximum_roads = 1
    maximum_lanes = 3
    minimum_lanes = 2
    road_number = np.random.randint(1,maximum_roads+1)
    roads = []
    for e in range(road_number):

        points = create_road(np.random.randint(-90,90), np.random.randint(50,80)/100., initial_point)
        road = Road(points, lane_width = 4., lanes = np.random.randint(minimum_lanes,maximum_lanes+1))
        roads.append(road)
    #world = World(roads = [road1, road2])

    #world = World(roads = [road2], dt = 0.1)
    world = World(roads = roads, dt = 0.05)
    return world
def normalize(a):
    return (a -np.mean(a))/np.std(a)
def rotation_matrix(angle):
    return np.array(([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]]))


## 2. Define useful classes (Game, World, Agent, AI)

In [182]:
class Game():
    def __init__(self, scale, width, height):
        self.gameDisplay = pygame.display.set_mode((width, height))
        self.gameDisplay.fill(white)
        self.clock = pygame.time.Clock()
        self.scale = scale
        self.width = width
        self.height = height
    
    def draw_roads(self,roads):
        self.gameDisplay.fill(white)
        # brown ground
        #for road in roads:
        #    for lane in road.lanes:
        #        pts = (lane * self.scale + np.array([self.width/2, self.height/2])).astype(int)
        #        pygame.draw.lines(self.gameDisplay, (160,82,45), False, pts.tolist(),  int(self.scale*(road.lane_width+1.)))
        for road in roads:
            for lane in road.lanes:
                pts = (lane * self.scale + np.array([self.width/2, self.height/2])).astype(int)
                pygame.draw.lines(self.gameDisplay, (100,100,100), False, pts.tolist(),  int(self.scale*(road.lane_width+1.)))
                pygame.draw.lines(self.gameDisplay, (200,255,200), False, pts.tolist())
        pygame.draw.lines(self.gameDisplay, (0,0,0), False, np.array([np.linspace(0,self.width,10), np.ones(10)*self.height/2]).T.tolist())
                
    def draw_line(self, p1, p2, color):
        vertices = np.array([p1,p2])
        pygame.draw.lines(self.gameDisplay, color, False, (vertices*self.scale+np.array([self.width/2, self.height/2])).tolist())
                
                
    def car(self, vertices, color):

        #x,y,psi,_ = state 
        
        #gameDisplay.fill(white)
        #collision_im = pygame.Surface((int(collision_rectangle[0]*self.scale), (int(collision_rectangle[1]*self.scale))))                              
        #carImg = pygame.image.load('car.png')
        #carImg = pygame.transform.rotozoom(carImg, 0, 0.02)
        #carImg = pygame.transform.scale(carImg, collision_im.get_size())
        #collision_im.blit(carImg,(0,0))
        #pygame.draw.rect(collision_im,blue,(0,0) + collision_im.get_size(), 2)
        #collision_im = collision_im.convert_alpha()
        
        pygame.draw.polygon(self.gameDisplay, color, (vertices*self.scale+np.array([self.width/2, self.height/2])).tolist())
        
        #im = pygame.transform.rotozoom(collision_im, -psi*180./3.14159, 1.)
        #self.gameDisplay.blit(im, (x*self.scale+self.width/2.,y*self.scale+self.height/2))
    def display_stats(self, messages):
        black = (0, 0, 0) 
        white = (255, 255, 255) 
        font = pygame.font.Font('freesansbold.ttf', 12)
        h = 0.8
        for m in messages:
            text = font.render(m, True, black, white)
            textRect = text.get_rect()
            textRect.center = (int(self.width*0.25), int(self.height*h)) 
            self.gameDisplay.blit(text, textRect) 
            h += 0.05
class Road():
    def __init__(self, points, lane_width, lanes):
        self.points = points
        self.lane_width = lane_width
        self.number_of_lanes = lanes
        self.lanes = []
        self.get_lanes()
        self.boundary = []
        self.get_road_boundary_polygon()
        self.lines = [geom.LineString(lane) for lane in self.lanes]
        
        
    def get_lanes(self):
        #pygame.draw.lines(self.gameDisplay, (100,100,100), False, show_road.tolist(), int(self.scale*road.lane_width*road.lanes))
        pts = self.points
        for lane in range(self.number_of_lanes):
            #pygame.draw.lines(self.gameDisplay, (100,100,100), False, pts.tolist(),  int(self.scale*road.lane_width+5))
            #pygame.draw.lines(self.gameDisplay, (200,255,200), False, pts.tolist())
            #poly = geom.Polygon(pts)
            # Create offset airfoils, both inward and outward
            #out_poly = poly.buffer(self.scale*road.lane_width/2) 
            #pts = np.array(out_poly.exterior)
            #poly = geom.polygon.LinearRing(pts)
            pts = interpolate_points(pts)
            self.lanes.append(pts)
            poly = geom.LineString(pts)

            # Create offset airfoils, both inward and outward
            out_poly = poly.parallel_offset(self.lane_width, side="right", resolution=16, 
                                                         join_style=2, mitre_limit=1)
            pts = np.array(out_poly)
            if pts[0][0] > pts[-1][0]:
                pts = pts[::-1]
    def get_road_boundary_polygon(self):
        left_side = np.array(geom.LineString(self.lanes[0]).parallel_offset(self.lane_width/2. + 1., side="left", resolution=16, join_style=2, mitre_limit=1))
        right_side = np.array(geom.LineString(self.lanes[-1]).parallel_offset(self.lane_width/2. + 1, side="right", resolution=16, join_style=2, mitre_limit=1))
        
        if left_side[0][0] > left_side[-1][0]:
            left_side = left_side[::-1]
        if right_side[0][0] > right_side[-1][0]:
            right_side = right_side[::-1]
        
        boundary = np.concatenate([left_side, right_side[::-1], np.array([left_side[0]])])
        self.boundary = geom.Polygon(boundary)
    
    def point_within_road(self, p):
        return geom.Point(p).within(self.boundary)
    
    def project_point(self, point, lane_number):
        point_on_line = self.lines[lane_number].interpolate(self.lines[lane_number].project(geom.Point(point[0], point[1])))
        return [point_on_line.x, point_on_line.y]
    def get_heading(self, point, lane_number):
        l = np.array(self.lines[lane_number])
        i = find_index(0, len(l) ,point, l)
        if i == len(l)-1:
            return l[i] - l[i-1]
        return l[i+1] - l[i]
    
    
    
class World():
    def __init__(self, roads, dt):
        self.total_agents_created = 0
        self.roads = roads
        self.agents = []
        self.dt = dt
        self.t = 0
        self.generate_road_boundary()
        self.laser_number = 24
        self.laser_max_range = 20.
        self.lane_map = [[{} for lane in range(road.number_of_lanes)] for road in self.roads]
        self.ai_created = False
        self.end_experiment = False
        self.completion_percentage = 0
        
    def generate_road_boundary(self):
        union = geom.Polygon()
        for road in self.roads:
            union = union.union(road.boundary)
        self.total_boundary = union
    def is_agent_out_of_track(self, vertices):
        res = False
        for v in vertices:
            is_on_road = False
            for road in self.roads:
                if road.point_within_road(v):
                    is_on_road = True
            if not is_on_road:
                return True
        return res
    
    def check_ai_crash(self):
        for agent in self.agents:
            if agent.agent_id == 0:
                ai_vertices = agent.vertices_polygon
                for v in agent.vertices:
                    if not geom.Point(v).within(self.total_boundary):
                        return True
        for agent in self.agents:
            if agent.agent_id != 0:
                if agent.vertices_polygon.intersects(ai_vertices):
                    return True
        return False
    
    def get_laser_readings(self):
        # find laser candidates
        laser_number = self.laser_number
        laser_angles = np.linspace(0, 2*np.pi, laser_number)
        for agent in self.agents:
            if agent.agent_id == 0:
                xc,yc,psic,vc = agent.state

        vectors = np.array([xc,yc]) + self.laser_max_range*np.array([np.cos(laser_angles+psic), np.sin(laser_angles+psic)]).T

        # road boundary
        free_road = self.total_boundary
        for agent in self.agents:
            if agent.agent_id != 0:
                free_road = free_road.difference(agent.vertices_polygon)

        # laser lines
        lines = [[[xc,yc],p.tolist()] for p in vectors]
        intersections = []
        for l in lines:
            intersection = free_road.intersection(geom.LineString(l))

            if intersection.type == 'LineString':
                intersection = np.array(intersection)[-1]
            elif intersection.type == 'MultiLineString':
                intersection = np.array(intersection[0])[-1]
            #    intersection = np.array(intersection.boundary)[0].tolist()
            else:
                print('ERROR')
            #    intersection = np.array(intersection).tolist()

            intersections.append(intersection)
        laser_readings = np.linalg.norm(np.array(intersections) - np.array([xc,yc]), axis = 1)
        return intersections, laser_readings
    def update_lane_map(self):
        
        self.lane_map = [[{} for lane in range(road.number_of_lanes)] for road in self.roads]
        for agent in self.agents:
            road = agent.control.desired_road
            lane = agent.control.desired_lane
            x,y,psi,v = agent.state
            point = [x,y]
            l = np.array(self.roads[road].lines[lane])
            i = find_index(0, len(l) ,point, l)
            if agent.agent_id == 0:
                self.completion_percentage = float(i) / (0.7*len(l))
                
            if  float(i) / (0.7*len(l)) > 1.1:
                
                self.agents.remove(agent)
            else:
                self.lane_map[road][lane][agent.agent_id] = i
            
    def generate_agent(self, ai = False):
        starting_id = 20
        minimum_distance = 30.
        lane_candidates = []
        max_lanes = max([road.number_of_lanes for road in self.roads])
        joint_lanes_min_id = [1000 for e in range(max_lanes)]
        
        for road in self.lane_map:
            for lane_number, lane in enumerate(road):
                if len(lane) > 0:
                    min_id = min([a for a in lane.values()])
                    if joint_lanes_min_id[lane_number] > min_id:
                        joint_lanes_min_id[lane_number] = min_id

        for road_number, road in enumerate(self.lane_map):
            for lane_number, lane in enumerate(road):
                if joint_lanes_min_id[lane_number] - starting_id >= minimum_distance:
                    lane_candidates.append((road_number,lane_number))

        if len(lane_candidates) > 0:
            (road,lane) = lane_candidates[np.random.randint(len(lane_candidates))]
            x,y = self.roads[road].lanes[lane][starting_id] + np.random.randn(2)
            psi = np.random.randn(1)[0] / 5
            v = 3.5 + np.random.randn(1)[0]
            state = np.array([x,y,psi,v])
            self.total_agents_created += 1
            if ai:
                color = [0,0,255]
                agent_id = 0
                collision_rectangle = (np.array([4,2.5]) + np.random.randn(2)/ 10.)*(1+np.random.randn(1)[0]*0.1)
                agent = Agent(agent_id, state, collision_rectangle, self.dt)
                agent.control.desired_road = road
                agent.control.desired_lane = lane
                #color[road] += 150
                #color[lane] += 80.
                agent.color = color
                self.ai_created = True
                agent.control.is_ai = True
                
            else:
                color = [0,255,0]
                
                agent_id = self.total_agents_created
                collision_rectangle = (np.array([4,2.5]) + np.random.randn(2)/ 10.)*(1+np.random.randn(1)[0]*0.1)
                agent = Agent(agent_id, state, collision_rectangle, self.dt)
                agent.control.desired_road = road
                agent.control.desired_lane = lane
                #color[road] += 150
                #color[lane] += 80.
                agent.color = color
            self.agents.append(agent)
            
    def get_agent_lane_pairs(self):
        agent_lane_pairs = []
        agent_indexes = {}
        for agent_index, agent in enumerate(self.agents):
            agent_indexes[agent.agent_id] = agent_index
        
        for road_number, road in enumerate(self.lane_map):
            for lane_number, lane in enumerate(road):
                for agent_id in lane:
                    lane_point = self.roads[road_number].lanes[lane_number][lane[agent_id]]
                    x,y,psi,v = self.agents[agent_indexes[agent_id]].state
                    agent_lane_pairs.append([np.array([x,y]), lane_point])
        return agent_lane_pairs
    
    def get_stats(self):
        messages = []
        messages.append('Number of agents: {}'.format(len(self.agents)))
        messages.append('Completion percentage: {}%'.format(np.round(self.completion_percentage*100, 2)))
        return messages
        
    def step(self, visualize = True):
        global game, AI
        out_of_track = False
        if visualize: game.draw_roads(self.roads)
        if self.t > 1.:
            #agents[0].control.desired_lane = 0
            self.generate_agent()
            self.update_lane_map()

        if self.t > 2. and not self.ai_created:
            self.generate_agent(ai = True)
            self.update_lane_map()
        for agent in self.agents:
            u0 = agent.control.select_driving_action(agent.state, agent.action)
            agent.action = u0
            x0 = agent.state
            agent.update_state(agent.control.get_next_state(agent.state,u0))
            if visualize: game.car(agent.vertices, agent.color)

            #if self.is_agent_out_of_track(agent.vertices):
            #    out_of_track = True
        if visualize:
            agent_lane_pairs = self.get_agent_lane_pairs()
            for a in agent_lane_pairs:
                p1 = a[0]
                p2 = a[1]
                game.draw_line(p1,p2, (0,0,200))

        if self.ai_created:
            ai_crashed = self.check_ai_crash()
            if ai_crashed:
                AI.crashed = True
                self.end_experiment = True
            intersections, laser_readings = self.get_laser_readings()
            for agent in self.agents:
                if agent.agent_id == 0:
                    x,y,psi,v = agent.state
                    vertices = agent.vertices
                    color = agent.color
                    action = agent.action
            if visualize:
                for laser_point in intersections:
                    p1 = [x,y]
                    p2 = laser_point
                    game.draw_line(p1,p2, (255,0,0))
                game.car(vertices, color)

            if AI.completion_percentage == -1:
                AI.memory.initialize_experiment()
                AI.completion_percentage = self.completion_percentage
            else:
                # reward
                if AI.crashed:
                    reward = -5
                    self.end_experiment = True
                elif self.completion_percentage > 1:
                    reward = +1
                    self.end_experiment = True
                else:
                    reward = self.completion_percentage - AI.completion_percentage
                    AI.completion_percentage = self.completion_percentage
                    
                # state
                state = laser_readings
                # action
                action = action

                AI.memory.add_to_memory(state, action, reward)
                

        # display map
        if visualize:
            game.display_stats(self.get_stats())
            pygame.display.update()
        # time update
        self.t += self.dt
        #time.sleep(self.dt/2)


class Control():
    def __init__(self, collision_rectangle, dt, mode):
        # car parameters
        self.collision_rectangle  = collision_rectangle
        car_length = collision_rectangle[0]
        self.lf = car_length*(0.65)/2
        self.lr = car_length*(0.65)/2
        self.dt = dt
        # control variables
        self.desired_road = 0
        self.desired_lane = 0
        self.desired_direction = 1
        self.delta_rate = 0.25
        self.max_delta = 0.8
        self.ey_history = [0,0,0,0,0]
        self.mode = mode
        self.is_ai = False
        
    def select_driving_action(self, state, old_action):
        global world
        
        # rarity: change desired lane
        if not self.is_ai:
            change_lane_frequency = 1./500 # 1 time every 1000 timesteps on average
            if np.random.randint(1,int(1./change_lane_frequency)) == 10:
                lane_number = world.roads[self.desired_road].number_of_lanes
                current_lane = self.desired_lane
                if lane_number == 1:
                    new_lane = current_lane
                    
                else:
                    if current_lane == 0:
                        new_lane = 1
                    elif current_lane == lane_number-1:
                        new_lane = current_lane-1
                    else:
                        new_lane = current_lane + np.random.choice([-1,+1])
                self.desired_lane = new_lane
        
        old_delta = old_action[0]
        if self.mode == 'PID':
            # PID
            kp = 0.2
            kd = 0.9
            ki = 0.2
            old_ey = self.ey_history[-1]

            ey, e_heading = self.position_error(state) * self.desired_direction
            self.ey_history =  self.ey_history[1:] + [ey]  
            I = ki * np.sum(self.ey_history)/len(self.ey_history)
            delta = -kp*ey - kd*(ey - old_ey)/self.dt - I
            delta = np.clip(delta, old_delta-self.delta_rate, old_delta+self.delta_rate)
            delta = np.clip(delta, -self.max_delta, self.max_delta)
            action = old_action
            action[0] = delta
            return action
        if self.mode == 'simple_MPC':
            timestep_horizon = 3
            Ky = 1/2.**2
            Kang = 1/0.9**2
            
            
            delta_min = max(-self.max_delta, old_delta -self.delta_rate)
            delta_max = min(self.max_delta, old_delta + self.delta_rate)
            delta_candidates = np.linspace(delta_min, delta_max, 8)
            min_error = 100000
            min_delta = old_delta
            #print('AAAAAAAAAAAA')
            for delta in delta_candidates:
                #print('Delta: {}'.format(delta))
                error = 0
                action = [delta, 0]
                s = state
                for t in range(timestep_horizon):
                    #print('State {} = {}'.format(t, s))
                    s = self.get_next_state(s, action)
                    ey, e_heading = self.position_error(s)
                    
                    error += Ky*ey**2 + Kang*e_heading**2
                    #print('State {} = {}, Ey = {}'.format(t, s, ey))
                #print(delta, error)
                if error < min_error:
                    min_error = error
                    min_delta = delta
                    
            action = old_action
            action[0] = min_delta
            return action
                    
                
        
    def position_error(self, state):
        global world
        x,y,psi,v = state
        
        point_on_line = world.roads[self.desired_road].project_point([x,y], self.desired_lane)
        road_heading = world.roads[self.desired_road].get_heading([x,y], self.desired_lane)
        psid = np.arctan2(road_heading[1], road_heading[0])
        xd, yd = point_on_line
        ey = -(x-xd)*np.sin(psid) + (y-yd)*np.cos(psid)
        #return np.linalg.norm([x-xd,y-yd])
        e_heading = min((psid -psi)%(2*np.pi), (psi -psid)%(2*np.pi))
        
        return ey, e_heading
        
    def get_next_state(self,x0,u0): 
        x,y,psi,v = x0
        delta, ax = u0
        lf, lr, dt = [self.lf, self.lr, self.dt]

        beta = np.arctan(lr/(lf+lr)*np.tan(delta))
        x_n = x + dt*(v * np.cos(psi+beta))
        y_n = y + dt*(v * np.sin(psi+beta))
        psi_n = psi + dt*(v / lr * np.sin(beta))
        v_n = v + dt*ax

        state = np.array([x_n, y_n, psi_n, v_n])
        return state
    

        
class Agent():
    def __init__(self, agent_id, initial_state, collision_rectangle, dt):
        self.agent_id = agent_id
        self.state = initial_state
        self.action = [0,0]
        self.vertices = vertices_from_state(self.state, collision_rectangle)
        self.vertices_polygon = geom.Polygon(self.vertices)
        self.crashed = False
        self.collision_rectangle = collision_rectangle
        self.color = (0,0,255)

        # control
        #self.control = Control(collision_rectangle, dt, 'simple_MPC')
        self.control = Control(collision_rectangle, dt, 'PID')
    def update_state(self,state):
        self.state = state
        self.vertices = vertices_from_state(state, self.collision_rectangle)
        self.vertices_polygon = geom.Polygon(self.vertices)
        


class AI_class:
    def __init__(self):
        # EXPERIENCE COLLECTION
        self.memory = Memory()
        self.completion_percentage = -1
        self.crashed = False
        
        # DL MODEL
        
    def reset_experiment(self):
        self.completion_percentage = -1
        self.crashed = False
class Memory:
    def __init__(self):
        self.experiments = []

    def initialize_experiment(self):
        self.experiments.append(Experiment())
    
    def add_to_memory(self,state, action, reward):
        self.experiments[-1].rewards.append(reward)
        self.experiments[-1].states.append(1./(0.2+state))
        self.experiments[-1].actions.append(action)
class Experiment:
    def __init__(self):
        self.rewards = []
        self.states = []
        self.actions = []

## 3. Simulate experiments

In [183]:
width = 800
height = 600
scale = 10
white = (255,255,255)


game = Game(scale, width, height)
pygame.init()
AI = AI_class()

experiment_number = 10
for e in range(experiment_number):

    world = initialize_world()
    while world.t < 300. and not world.end_experiment:
        world.step(visualize = True)
    AI.reset_experiment()



In [ ]:
for i,e in enumerate(AI.memory.experiments):
    print('Exp: ~{}'.format(i))
    print(e.rewards[-1], max(e.states[-1]))
    

# Define Soft Actor Critic Networks

In [26]:
class SoftQNetwork():
    def __init__(self):
        self.model = {}
        self.create_model()
    def create_model(self):
        # Model parameters
        H = 64
        A = 2 # delta and acceleration
        S = 24 # laser readings
        # Model weights
        self.model['W1'] = np.random.randn(H,S+A) / np.sqrt(S+A)
        self.model['W2'] = np.random.randn(H,H) / np.sqrt(H)
        self.model['W3'] = np.random.randn(1,H) / np.sqrt(H)

        self.model['b1'] = np.random.randn(H)
        self.model['b2'] = np.random.randn(H)
        self.model['b3'] = np.random.randn(1)
    def forward(self,x):
        h1 = np.dot(self.model['W1'], x)+self.model['b1']
        h1[h1<0] = 0
        h2 = np.dot(self.model['W2'], h1)+self.model['b2']
        h2[h2<0] = 0
        y = np.dot(self.model['W3'], h2)+self.model['b3']
        return h1,h2,y
    def backprop(self,y,dy,x,h1,h2):
        # layer 3
        db3 = dy
        dw3 = np.outer(dy, h2)
        dh2 = np.dot(dy, self.model['W3'])
        dh2[h2==0] = 0 # relu
        # layer 2
        db2 = dh2
        dw2 = np.outer(dh2, h1)
        dh1 = np.dot(dh2, self.model['W2'])
        dh1[h1==0] = 0# relu
        # layer 1
        db1 = dh1
        dw1 = np.outer(dh1, x)
        dx = np.dot(dh1, self.model['W1'])
        
        return [db3, dw3, dh2, db2, dw2, dh1, db1, dw1, dx]

        

In [27]:
softq = SoftQNetwork()
# example
x = np.random.randn(26)
dy = np.random.randn(1)
# forward
h1,h2,y = softq.forward(x)
# backprop
[db3, dw3, dh2, db2, dw2, dh1, db1, dw1, dx] = softq.backprop(y,dy,x,h1, h2)

In [31]:
class GaussianPolicy():
    def __init__(self):
        self.model = {}
        self.create_model()
    def create_model(self):
        # Model parameters
        H = 64
        A = 2 # delta and acceleration
        S = 24 # laser readings
        # Model weights
        self.model['W1'] = np.random.randn(H,S) / np.sqrt(S)
        self.model['W2'] = np.random.randn(H,H) / np.sqrt(H)
        self.model['W_mean'] = np.random.randn(A,H) / np.sqrt(H)
        self.model['W_log_std'] = np.random.randn(A,H) / np.sqrt(H)

        self.model['b1'] = np.random.randn(H)
        self.model['b2'] = np.random.randn(H)
        self.model['b_mean'] = np.random.randn(A)
        self.model['b_log_std'] = np.random.randn(A)
    
    def forward(self,state):
        h1 = np.dot(self.model['W1'], state)+self.model['b1']
        h1[h1<0] = 0
        h2 = np.dot(self.model['W2'], h1)+self.model['b2']
        h2[h2<0] = 0
        mean = np.dot(self.model['W_mean'], h2)+self.model['b_mean']
        log_std = np.dot(self.model['W_log_std'], h2)+self.model['b_log_std']
        
        return h1,h2,mean,log_std
    

In [ ]:
    def sample(self, state, epsilon=1e-6):
        mean, log_std = self.forward(state)
        std = log_std.exp()
        normal = Normal(mean, std)
        
        # to obtain actions, we sample a z-value from the obtained Gaussian distribution
        # later, we will take the hyperbolic tangent of the z value to obtain our action. 
        # (see below in the post).
        z = normal.rsample()
        
        # we modify the log_pi computation as explained in the Haarnoja et al. paper
        log_pi = (normal.log_prob(z) - torch.log(1 - (torch.tanh(z)).pow(2) + epsilon)).sum(1, keepdim=True)

        return mean, std, z, log_pi

In [32]:
gaussian_policy = GaussianPolicy()

In [35]:
x = np.random.randn(24)
h1,h2,mean,log_std = gaussian_policy.forward(x)

In [36]:
std = np.exp(log_std)


In [52]:
np.random.normal([4., -3.], [0.1, 20])

array([4.15601359, 0.01399136])

In [ ]:
z = np.random.normal(mean, std)
log_pi = ()

In [45]:
np.random.normal(mean, std)

array([0.8721962 , 1.36389831])

In [38]:
mean

array([0.33464858, 1.04502724])

In [37]:
std

array([0.99535953, 0.23848904])

In [ ]:
 

class GaussianPolicy(nn.Module):

    def __init__(self, num_inputs, num_actions,
                 hidden_size=256, init_w=3e-3, log_std_min=-20, log_std_max=2):
        super(GaussianPolicy, self).__init__()
        self.log_std_min = log_std_min
        self.log_std_max = log_std_max

        self.linear1 = nn.Linear(num_inputs, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)

        # we have one network head that outputs the mean of the Gaussian distribution for every action.
        self.mean_linear = nn.Linear(hidden_size, num_actions)
        self.mean_linear.weight.data.uniform_(-init_w, init_w)
        self.mean_linear.bias.data.uniform_(-init_w, init_w)
        
        # we have another network head that outputs the log(covariance) of the Gaussian distribution for every action.
        self.log_std_linear = nn.Linear(hidden_size, num_actions)
        self.log_std_linear.weight.data.uniform_(-init_w, init_w)
        self.log_std_linear.bias.data.uniform_(-init_w, init_w)

    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))

        mean = self.mean_linear(x)
        log_std = self.log_std_linear(x)
        log_std = torch.clamp(log_std, self.log_std_min, self.log_std_max)
        
        # as explained, the neural network outputs the mean and the (log of the) covarance.
        return mean, log_std


